In [6]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import urllib.parse
import time
from geopy.geocoders import Nominatim, GoogleV3
from retrying import retry
from sqlalchemy import create_engine

In [2]:
def passportXMLtoDF(filePath):
    tree = ET.parse(filePath)
    
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    
    root = tree.getroot()
    document = root[0]
    folder = document[5]
    
    businesses = []
    for placemark in folder.findall("kml:Placemark", ns):
        business = {
            'name': placemark.find('kml:name', ns).text,
            'cdata_description': placemark.find('kml:description', ns).text
        }
        extended_data = placemark.find('kml:ExtendedData', ns)
        for child in extended_data:
            attribute = child.attrib['name']
            value = child[0].text
            if attribute == 'Address':
                business['address'] = value
            elif attribute == 'Phone Number':
                business['phone_number'] = value
            elif attribute == 'Drink Offering':
                business['drink_offering'] = value
            elif attribute == 'Website':
                business['website'] = value
            else:
                raise RuntimeError("A key appeared in the Placemark/ExtendedData that we were not looking for")
        businesses.append(business)


    return pd.DataFrame(businesses)

In [3]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
def geolocate(address):
    print('working on ', address, '...')
    # geolocator = Nominatim()
    geolocator = GoogleV3()

    try:
        location = geolocator.geocode(address)
    except:
        print('\tTimeout')
        raise Exception('Time out')

    if location == None:
        print('\tEmpty Response')
        raise Exception('Empty Response')

    return location.latitude, location.longitude

In [4]:
df = passportXMLtoDF('./data/2018SummerDenverPassportMap.kml')

In [5]:
df['lat'], df['lon'] = zip(*df['address'].map(geolocate))

working on  501 E 17th Ave, DENVER, CO 80203 ...
working on  2134 larimer st, denver, CO 80205 ...
	Timeout
working on  2134 larimer st, denver, CO 80205 ...
	Timeout
working on  2134 larimer st, denver, CO 80205 ...
working on  1539 S Broadway, Denver, CO 80210 ...
working on  2706 Larimer St, Denver, CO 80205 ...
working on  620 E 6th Ave, Denver, CO 80203 ...
	Timeout
working on  620 E 6th Ave, Denver, CO 80203 ...
working on  470 Broadway, Denver, CO 80203 ...
working on  3126 Larimer St, Denver, Co 80205 ...
working on  1879 S Acoma St, Denver, Co 80223 ...
working on  5126 E Colfax Ave, Denver, CO 80220 ...
	Timeout
working on  5126 E Colfax Ave, Denver, CO 80220 ...
working on  1420 Larimer St, Denver, CO 80202 ...
working on  4000 Tennyson St, Denver, CO 80212 ...
	Timeout
working on  4000 Tennyson St, Denver, CO 80212 ...
working on  4450 w 38th ave, Denver, CO 80221 ...
working on  1701 Wynkoop St, Denver, CO 80202 ...
working on  3350 Brighton Blvd, Unit 130, Denver, CO 8021

In [13]:
#df.loc[(df['latitude'] == 0) & (df['longitude'] == 0)]
df.rename(index=str, columns={"lat": "latitude", "lon": "longitude"})

,address,cdata_description,drink_offering,name,phone_number,website,latitude,longitude
0,"501 E 17th Ave, DENVER, CO 80203","Address: 501 E 17th Ave, DENVER, CO 80203<br>P...",draft beer or highballs,ace eat serve,(303)800-7705,http://www.aceeatserve.com,39.743558,-104.980734
1,"2134 larimer st, denver, CO 80205","Address: 2134 larimer st, denver, CO 80205<br>...",local can beers or mai tais,aloy modern thai,(303)379-9759,http://www.aloymodernthai.com/,39.754304,-104.990611
2,"1539 S Broadway, Denver, CO 80210","Address: 1539 S Broadway, Denver, CO 80210<br>...",drafts,alternation brewing company,(303)502-5340,http://www.alternationbrewing.com,39.688657,-104.987895
3,"2706 Larimer St, Denver, CO 80205","Address: 2706 Larimer St, Denver, CO 80205<br>...",single barrel old fashioned,american bonded,(303)942-1201,http://americanbonded.com/maybe,39.759451,-104.983678
4,"620 E 6th Ave, Denver, CO 80203","Address: 620 E 6th Ave, Denver, CO 80203<br>Ph...",carboy wine on tap,angelo's taverna,(303)744-3366,http://angelostaverna.com,39.725433,-104.978999
5,"470 Broadway, Denver, CO 80203","Address: 470 Broadway, Denver, CO 80203<br>Pho...",drafts,banded oak brewing company,(720)479-8033,http://bandedoakbrewing.com,39.723769,-104.987010
6,"3126 Larimer St, Denver, Co 80205","Address: 3126 Larimer St, Denver, Co 80205<br>...",aperol spritz,bar fausto,(720)445-9691,http://www.barfausto.com,39.763458,-104.978577
7,"1879 S Acoma St, Denver, Co 80223","Address: 1879 S Acoma St, Denver, Co 80223<br>...",handcrafted cocktails,bear creek distillery,(303)955-4638,http://bearcreekdistillery.com,39.682570,-104.989132
8,"5126 E Colfax Ave, Denver, CO 80220","Address: 5126 E Colfax Ave, Denver, CO 80220<b...",bartender's choice or 400 yards due north,bellwether,(720)398-9251,http://bellwetherdenver.com,39.739918,-104.927425
9,"1420 Larimer St, Denver, CO 80202","Address: 1420 Larimer St, Denver, CO 80202<br>...","beer, cocktails, or wines by the glass",bistro vendôme,(303)825-3232,http://bistrovendome.com,39.747605,-104.999161


In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/2018_passport_data.db', echo=False)
df.to_sql('passport', con=engine)

In [16]:
engine.execute("SELECT * FROM passport").fetchall()

[(0, '501 E 17th Ave, DENVER, CO 80203', 'Address: 501 E 17th Ave, DENVER, CO 80203<br>Phone Number: (303)800-7705<br>Drink Offering: draft beer or highballs<br>Website: http://www.aceeatserve.com', 'draft beer or highballs', 'ace eat serve', '(303)800-7705', 'http://www.aceeatserve.com', 39.7435577, -104.9807338),
 (1, '2134 larimer st, denver, CO 80205', 'Address: 2134 larimer st, denver, CO 80205<br>Phone Number: (303)379-9759<br>Drink Offering: local can beers or mai tais <br>Website: http://www.aloymodernthai.com/', 'local can beers or mai tais ', 'aloy modern thai', '(303)379-9759', 'http://www.aloymodernthai.com/', 39.7543043, -104.9906115),
 (2, '1539 S Broadway, Denver, CO 80210', 'Address: 1539 S Broadway, Denver, CO 80210<br>Phone Number: (303)502-5340<br>Drink Offering: drafts<br>Website: http://www.alternationbrewing.com', 'drafts', 'alternation brewing company', '(303)502-5340', 'http://www.alternationbrewing.com', 39.688657, -104.987895),
 (3, '2706 Larimer St, Denver, C